In [ ]:
import os
import pandas as pd
import re

# List of colleges
colleges = ['education', 'engineering', 'chs', 'liberalarts', 'nursing', 'science', 'pharmacy', 'business', 'graduate', 'extendeduniversity']

# Regular expression pattern to extract folder name
folder_pattern = re.compile(r'https?://[^/]+/[^/]+/([^/]+)/')

# Create a dictionary to store links based on folder names
link_dict = {}

# Iterate through each college
for college in colleges:
    input_excel_path = f'Documents/Excel_Files/{college}.xlsx'

    # Load the Excel file using pandas with column 'A' as index
    df = pd.read_excel(input_excel_path, index_col=0)

    # Iterate through each link in column 'A'
    for link in df.index:
        match = folder_pattern.search(link)
        if match:
            folder_name = match.group(1)
            if folder_name not in link_dict:
                link_dict[folder_name] = []
            link_dict[folder_name].append(link)

    # Create a new Excel file for each folder
    output_folder = 'Documents/Excel_Files/'
    os.makedirs(output_folder, exist_ok=True)

    for folder_name, links in link_dict.items():
        output_excel_path = os.path.join(output_folder, f'{college}-{folder_name}.xlsx')
        links_df = pd.DataFrame({'Links': links, 'Template': ['No Carousel'] * len(links)})
        links_df.to_excel(output_excel_path, index=False)

    print("Separation completed.")


In [1]:
import os
import pandas as pd

# Input folder containing Excel files
input_folder = 'Documents/Excel_Files/'

# Output Excel file path
output_excel_path = 'Documents/Summary.xlsx'

# Get a list of all Excel files in the input folder
excel_files = [file for file in os.listdir(input_folder) if file.endswith('.xlsx')]

# Create a DataFrame to store the data
data = {'Folder': [os.path.splitext(file)[0] for file in excel_files],
        'BR Done': [''] * len(excel_files),
        'Header[Date]': [''] * len(excel_files)}

df = pd.DataFrame(data)

# Write the DataFrame to the output Excel file
df.to_excel(output_excel_path, index=False)

print("Summary Excel file created.")


Summary Excel file created.


In [11]:
import requests
import csv
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote
from docx.shared import RGBColor, Inches, Pt
from docx import Document as WordDocument
import docx
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from PIL import Image
import io
import base64
import time
import urllib.parse
import urllib

In [51]:
    def identify_section(link, page_link, page_style):
        try:
            absolute_page_link = urljoin(link, page_link)
            response = requests.get(link)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                # Search for partial matches of the href attribute with page_link
                saved_link=page_link
                page_link=urllib.parse.unquote(page_link)  # Remove any URL encoding

                delimiter_characters = ['~', '%0D', '%']
                common_elements = []

                for delimiter in delimiter_characters:
                    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1')
                    print(page_link)
                    if delimiter in page_link:
                        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2')
                        print(page_link)
                        print(delimiter)

                        parts = page_link.split(delimiter)
                        
                        print(parts)
                        elements_part = [soup.find_all('a', href=lambda href: part in href) for part in parts]
                        print(f'Element parts are: {elements_part}')
                        if elements_part:
                            if not common_elements:
                                common_elements = elements_part
                            else:
                                common_elements = [element for element in common_elements if element in elements_part]
                        print(common_elements)
                    if common_elements:
                        element = common_elements[0][0]  # Returning the first common element found
                        break
                    else:
                        element = soup.find('a', href=lambda href: page_link in href)
                        print(element)
                print('here')
                if element is None:
                    for delimiter in delimiter_characters:
                        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1')
                        print(saved_link)
                        if delimiter in saved_link:
                            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2')
                            print(saved_link)
                            print(delimiter)

                            parts = saved_link.split(delimiter)

                            print(parts)
                            elements_part = [soup.find_all('a', href=lambda href: part in href) for part in parts]
                            print(f'Element parts are: {elements_part}')
                            if elements_part:
                                if not common_elements:
                                    common_elements = elements_part
                                else:
                                    common_elements = [element for element in common_elements if element in elements_part]
                        
                        if common_elements:
                            if common_elements[0]:
                                element = common_elements[0][0]  # Get the link from the second inner list
                            elif common_elements[1]:
                                element = common_elements[1][0]  # Get the link from the first inner list
                        else:
                            element = soup.find('a', href=lambda href: page_link in href)

                if element:
                    element_name = 'No_name'
                    element_name = element.get_text(strip=True)
                    
                    # Check for accordion or tab
                    accordion_div = element.find_parent('div', class_='collapsable collapse')
                    if accordion_div:
                        accordion_id = accordion_div.get('id')
                        if accordion_id:
                            element_name = f"{accordion_id}/{element_name}"

                    tab_div = element.find_parent('div', class_='tab-pane')
                    if tab_div:
                        tab_id = tab_div.get('id')
                        if tab_id:
                            element_name = f"{tab_id}/{element_name}"


                    if page_style == "No Carousel":

                        left_nav_element = element.find_parent('div', class_='leftSidebar')
                        if left_nav_element:
                            return "Left Navigation", element_name

                        main_element = element.find_parent('div', class_='rightSidebar')
                        if main_element:
                            return "Main Content", element_name   
                        
                        breadcrumb = element.find_parent(class_='breadcrumb')
                        if breadcrumb:
                            return "Main Content (Top small navigation)", element_name
                        
                        nav_element = element.find_parent('nav')
                        if nav_element:
                            # Find the outer ul element with class "nav navbar-nav" within the nav_element
                            outer_ul = nav_element.find('ul', class_='nav navbar-nav')

                            if outer_ul:
                                # Loop through <li> elements with class 'dropdown'
                                for li in outer_ul.find_all('li', class_='dropdown'):
                                    # Check if the element is a descendant of the current <li>
                                    if element in li.descendants:
                                        # Find the specific <a> element within the <li>
                                        dropdown_a = li.find('a', class_='dropdown-toggle')
                                        if dropdown_a:
                                            dropdown_name = dropdown_a.get_text(strip=True)
                                            return f"Main Navigation, tab name: {dropdown_name}", f"{dropdown_name}/{element_name}"
                                return "Main Navigation", element_name


                        footer_element = element.find_parent('footer')
                        if footer_element:
                            return "Footer", element_name

                        #Fullscreen
                        section_anchor_div = soup.find('div', class_='sectionAnchor')
                        if section_anchor_div:
                            section_id = section_anchor_div.get('id')
                            
                            if section_id:
                                return f"Main Content: Section ID: {section_id}" , element_name

                            else:
                                return f"Main Content: Section ID: No ID", element_name

                        
                        
                        sidr_container_element = element.find_parent('div', id='sidr-container')
                        if sidr_container_element:
                            return "Somewhere unknown", element_name
                    else:
                        return "TemplateUnknown","None"
                #Error case
                return "Unknown-notfound","None"  # If the element is not found in any of the relevant outer elements
            else:
                return "Unknown-pageerror","None"  # If there was an issue fetching the page
        except requests.exceptions.RequestException:
            return "Unknown-connectionerror","None"  # If there was a connection error or other issues


In [52]:
identify_section("https://www.utep.edu/liberalarts/cibs/resources/resources-on-the-web.html",
                 "https://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com",
                'No Carousel')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
http:/www.colonialmaiolica.com
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
http:/www.colonialmaiolica.com
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
http:/www.colonialmaiolica.com
None
here
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
https://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
https://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
https://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com
%0D
['https://www.utep.edu/cibs/resource/', 'http:/www.colonialmaiolica.com']
Element parts are: [[], [<a href="http://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com"><strong><u>http://www.colonialmaiolica.com</u></strong></a>]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
https://www.utep.edu/cibs/resource/%0Dhttp:/www.colonialmaiolica.com
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
https://www.utep.edu/cibs/resource/%0Dht

('Main Content', 'http://www.colonialmaiolica.com')

In [54]:
import requests

def check_urls(urls):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    for url in urls:
        try:
            print("Checking:", url)
            response = requests.get(url, headers=headers)
            print("Response:", response.status_code)
        except requests.exceptions.RequestException as e:
            print("Error:", e)

if __name__ == "__main__":
    urls_to_check = [
        "https://www.utep.edu/student-affairs/",
        "https://www.utep.edu/vpba/parking-and-transportation/",
        "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019",
        "https://www.utep.edu/human-resources/services/employment/index.html",
        "https://www.utep.edu/library/",
        "https://www.utep.edu/vpba/",
        "https://www.utep.edu/",
        "https://www.alaskaair.com/",
        "https://www.vivaaerobus.com/",
        "https://www.flyfrontier.com/",
        "https://visitelpaso.com/",
        "https://www.allegiantair.com/",
        "https://tarmexico.com/",
        "https://www.southwest.com/",
        "https://www.cbp.gov/travel",
        "https://www.delta.com/",
        "https://www.utep.edu/technologysupport/",
        "https://www.utep.edu/newsfeed/",
        "https://www.utep.edu/feedback",
        "https://www.volaris.com/",
        "https://elpasointernationalairport.com/",
        "https://goo.gl/maps/kaBASrmU8vz",
        "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/_Files/docs/HLS-Parking-Permit.pdf",
        "https://www.google.com/maps/dir/Aeropuerto+Internacional+de+Ciudad+Ju%C3%A1rez,+Aeropuerto,+Ciudad+Juarez,+Chihuahua,+Mexico/The+University+of+Texas+at+El+Paso,+Union+Building+East,+El+Paso,+TX/@31.6985343,-106.5395071,12z/am=t/data=!4m14!4m13!1m5!1m1!1s0x86e76759b0bafe3b:0xb9a91474653b6d95!2m2!1d-106.435702!2d31.635723!1m5!1m1!1s0x86e7585bc791653f:0x6ea37b84dbdc3238!2m2!1d-106.5033081!2d31.7716573!3e0?hl=en&shorturl=1",
        "https://goo.gl/maps/ba9qrEXjHv92",
        "https://www.google.com/maps/dir/El+Paso+International+Airport+(ELP),+6701+Convair+Rd,+El+Paso,+TX+79925/The+University+of+Texas+at+El+Paso,+Union+Building+East,+El+Paso,+TX/@31.7929349,-106.4811159,13z/am=t/data=!4m14!4m13!1m5!1m1!1s0x86e75a93bc660f7d:0x1f2f4f000097b549!2m2!1d-106.3824345!2d31.8053354!1m5!1m1!1s0x86e7585bc791653f:0x6ea37b84dbdc3238!2m2!1d-106.5033081!2d31.7716573!3e0?hl=en&shorturl=1",
        "https://apps.cbp.gov/bwt/mobile.asp?action=n&pn=2402",
        "https://bwt.cbp.gov/index.html",
        "https://www.oma.aero/es/pasajeros/ciudad-juarez/",
        "https://www.utep.edu/clery/",
        "https://events.utep.edu/",
        "https://www.volaris.com/",
        "https://www.vivaaerobus.com/ https://www.vivaaerobus.com/index.html",
        "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html",
        "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/",
        "https://www.utep.edu/human-resources/services/employment/index.html"
    ]
    
    check_urls(urls_to_check)


Checking: https://www.utep.edu/student-affairs/
Response: 200
Checking: https://www.utep.edu/vpba/parking-and-transportation/
Response: 200
Checking: https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019
Response: 200
Checking: https://www.utep.edu/human-resources/services/employment/index.html
Response: 200
Checking: https://www.utep.edu/library/
Response: 200
Checking: https://www.utep.edu/vpba/
Response: 200
Checking: https://www.utep.edu/
Response: 200
Checking: https://www.alaskaair.com/
Response: 403
Checking: https://www.vivaaerobus.com/
Response: 403
Checking: https://www.flyfrontier.com/
Response: 200
Checking: https://visitelpaso.com/
Response: 200
Checking: https://www.allegiantair.com/
Response: 403
Checking: https://tarmexico.com/
Response: 200
Checking: https://www.southwest.com/
Response: 200
Checking: https://www.cbp.gov/travel
Response: 200
Checking: https://www.delta.com/
Response: 403
Checking: https://www.utep.edu/technologysupport/
Response: 200
Chec

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_links(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        } 
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = []

            for link in soup.find_all('a', href=True):
                href = link.get('href')
                if href:
                    full_url = urljoin(url, href)
                    links.append(full_url)

            return links
        else:
            print(f'Failed to fetch content from {url}. Status code: {response.status_code}')
            return []
    except Exception as e:
        print(f'Error fetching content from {url}: {e}')
        return []

base_url = "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html"
links = extract_links(base_url)

for link in links:
    print(link)


https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html#mainContentArea
https://www.utep.edu/
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/registration/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/program/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/keynote-speakers/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-

In [2]:
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_links(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        http = urllib3.PoolManager()
        response = http.request('GET', url, headers=headers)

        if response.status == 200:
            soup = BeautifulSoup(response.data, 'html.parser')
            links = []

            for link in soup.find_all('a', href=True):
                href = link.get('href')
                if href:
                    full_url = urljoin(url, href)
                    links.append(full_url)

            return links
        else:
            print(f'Failed to fetch content from {url}. Status code: {response.status}')
            return []
    except Exception as e:
        print(f'Error fetching content from {url}: {e}')
        return []

def check_links(links):
    http = urllib3.PoolManager()

    for link in links:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
            }
            response = http.request('GET', link, headers=headers, timeout=3.0)
            print(f'{response.status} for {link}')
        except urllib3.exceptions.TimeoutError:
            print(f'Timeout for {link}')
        except Exception as e:
            print(f'Error accessing {link}: {e}')

base_url = "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html"
links = extract_links(base_url)

check_links(links)



200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html#mainContentArea
200 for https://www.utep.edu/
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/registration/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/program/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/keynote-speakers/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-20

In [3]:
import urllib3
import threading
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_links(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        http = urllib3.PoolManager()
        response = http.request('GET', url, headers=headers)

        if response.status == 200:
            soup = BeautifulSoup(response.data, 'html.parser')
            links = []

            for link in soup.find_all('a', href=True):
                href = link.get('href')
                if href:
                    full_url = urljoin(url, href)
                    links.append(full_url)

            return links
        else:
            print(f'Failed to fetch content from {url}. Status code: {response.status}')
            return []
    except Exception as e:
        print(f'Error fetching content from {url}: {e}')
        return []

def check_link(link):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        http = urllib3.PoolManager()
        response = http.request('GET', link, headers=headers, timeout=3.0)
        print(f'{response.status} for {link}')
    except urllib3.exceptions.TimeoutError:
        print(f'Timeout for {link}')
    except Exception as e:
        print(f'Error accessing {link}: {e}')

def main():
    base_url = "https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html"
    links = extract_links(base_url)

    max_threads = 10  # Define the maximum number of threads
    threads = []

    for link in links:
        thread = threading.Thread(target=check_link, args=(link,))
        thread.start()
        threads.append(thread)

        if len(threads) >= max_threads:
            for thread in threads:
                thread.join()
            threads = []

    # Wait for remaining threads to finish
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/program/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/keynote-speakers/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html#mainContentArea
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/registration/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/logistics/index.html
200 for https://www.utep.edu/
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-2019/index.html
200 for https://www.utep.edu/liberalarts/hispanic-linguistics-symposium-20